# Place for experimenting the progressive design

In [26]:
import yaml
import inspect
import importlib
import os,sys

sys.path.append('..')

import model_discovery.utils as U
from model_discovery.configs.gam_config import GAMConfig, GAMConfig_14M
from model_discovery.model.composer import GAUTree,GAUBase,GAUNode
# from model_discovery.evolution import BuildEvolution
import model_discovery.agents.flow.gau_utils
importlib.reload(model_discovery.agents.flow.gau_utils)
from model_discovery.agents.flow.gau_utils import check_and_reformat_gau_code

ckpt_dir = os.environ['CKPT_DIR']
lib_dir = U.pjoin(ckpt_dir, 'test_composer', 'lib')
test_tree = GAUTree('TestTree', None, None, None,None,lib_dir)

prompts_dir='../model_discovery/agents/prompts/'
gab_py = U.read_file(U.pjoin(prompts_dir,'gab_template.py'))
gam_py = U.read_file(U.pjoin(prompts_dir,'gam_prompt.py'))
GAU_TEMPLATE = U.read_file(U.pjoin(prompts_dir,'gau_template.py'))
GAU_BASE=inspect.getsource(GAUBase)


## Parsers

In [42]:
import json
import os,sys
import importlib

sys.path.append('..')

import model_discovery.agents.flow.gau_utils
importlib.reload(model_discovery.agents.flow.gau_utils)
from model_discovery.agents.flow.gau_utils import check_and_reformat_gau_code
import model_discovery.agents.prompts.prompts as P
import model_discovery.model.composer
importlib.reload(model_discovery.model.composer)
from model_discovery.model.composer import GAUTree,GAUBase,GAUNode
import model_discovery.utils
importlib.reload(model_discovery.utils)
import model_discovery.utils as U



code='''
# gau.py   # DO NOT CHANGE OR REMOVE THE MAKK HERE, KEEP IT ALWAYS THE FIRST LINE #

import torch
import torch.nn as nn

from model_discovery.model.utils.modules import GAUBase # DO NOT CHANGE THIS IMPORT STATEMENT #
from model_discovery.model.utils.modules import GAUBase # DO NOT CHANGE THIS IMPORT STATEMENT #
from model_discovery.model.utils.modules import gau_test # DO NOT CHANGE THIS IMPORT STATEMENT #

import torch.nn.functional as F

# YOU CAN IMPORT MORE MODULES HERE #

# YOU CAN DEFINE MORE CLASSES OR FUNCTIONS HERE #

# Placeholder classes for future implementation
class MemoryAccessUnit(nn.Module):
    def __init__(self, embed_dim, memory_size, device=None, dtype=None):
        super().__init__(embed_dim)

    def _forward(self, X, **Z):
        return X, {}

class DownsamplingUnit(nn.Module):
    def __init__(self, embed_dim, downsample_factor, device=None, dtype=None):
        super().__init__(embed_dim)

    def _forward(self, X, **Z):
        return X, {}


class XXX(GAUBase,object):
    def __init__(self, embed_dim: int, device=None, dtype=None, **kwargs):
        super().__init__(embed_dim)


class GAU(GAUBase): # DO NOT CHANGE THE NAME OF THIS CLASS
    """Generalized Autoregressive Block Unit
        Input:        X: (batch, seqlen, embed_dim), Z: {dict of all current intermediate variables}
        Output:       Y: (batch, seqlen, embed_dim), Z_: Optional, {dict of *new* intermediate variables to update the current Z}
        Constraints:  Causal, differentiable, parameter number, complexity, parallelizable
    """
    def __init__(self, embed_dim: int, device=None, dtype=None,**kwargs): # YOU CAN ADD MORE ARGUMENTS WITH OPTIONAL DEFAULT VALUES, BUT YOU HAVE TO HAVE embed_dim, device, dtype AS THE ARGUTMENTS #
        # argv: list of hyperparameters
        factory_kwargs = {"device": device, "dtype": dtype} # remember to pass it to all nn layers
        super().__init__(embed_dim) # DO NOT CHANGE THIS LINE #

        self.a1 = kwarg_all.get('a1', 1)
        self.a2: int = kwarg_all.get('a2', 2)

        # COMPLETING THE CODE HERE #
        self.token_scorer = TokenScoringGAU(embed_dim, **factory_kwargs)
        self.dual_path: GAUBase = DualPathGAU(embed_dim, **factory_kwargs)
        self.latent_attention: GAUBase = LatentAttentionGAU(embed_dim, **factory_kwargs)

        self.stages: nn.ModuleList = nn.ModuleList([CompressionStageGAU(embed_dim, device=
            device, dtype=dtype, **kwargs) for _ in range(2)])

        self.modelx=nn.ModuleList([
            nn.Conv2d(1,20,5),
            nn.ReLU(),
            LatentAttentionGAU(),
            nn.Conv2d(20,64,5),
            nn.ReLU(),
        ])
            

        self.model1 = (
          nn.Conv2d(1,20,5),
          nn.ReLU(),
          LatentAttentionGAU(),
          nn.Conv2d(20,64,5),
          nn.ReLU(),
          nn.Sequential(OrderedDict([
                ('conv1', nn.Conv2d(1, 20, 5)),
                ('relu1', nn.ReLU()),
                ('gau2', DualPathGAU()),
                ('conv2', nn.Conv2d(20, 64, 5)),
                ('relu2', nn.ReLU())
            ]))
        )

        self.model = nn.Sequential(
          nn.Conv2d(1,20,5),
          nn.ReLU(),
          LatentAttentionGAU(),
          nn.Conv2d(20,64,5),
          nn.ReLU(),
          nn.Sequential(OrderedDict([
                ('conv1', nn.Conv2d(1, 20, 5)),
                ('relu1', nn.ReLU()),
                ('gau2', DualPathGAU()),
                ('conv2', nn.Conv2d(20, 64, 5)),
                ('relu2', nn.ReLU())
            ]))
        )

        self.activations = nn.ModuleDict({
            'relu': nn.ReLU(),
            'lrelu': nn.LeakyReLU(),
            'gau3': CompressionStageGAU()
        })

        self.model2 = nn.Sequential(OrderedDict([
          ('conv1', nn.Conv2d(1,20,5)),
          ('relu1', nn.ReLU()),
          ('gau2', DualPathGAU()),
          ('conv2', nn.Conv2d(20,64,5)),
          ('relu2', nn.ReLU())
        ]))

    # YOU CAN ADD MORE FUNCTIONS HERE #


    def init_something(self):
        self.modelx[3]=LatentAttentionGAU()
        self.modelx.append(LatentAttentionGAU())
        self.modelx.insert(0,LatentAttentionGAU())
        self.activations['gau1']=LatentAttentionGAU()
        self.model2.add_module('gau4',LatentAttentionGAU())

        X=self.dual_path(X)
    
    def _forward(self, X, **Z): 

        # THE CODE HERE MUST BE COMPLETED #
        # Step 1: Score tokens
        X, Z = self.token_scorer(X, **Z)
        # Step 2: Route through dual paths
        # Step 3: Apply latent attention
        Y, Z = self.latent_attention(X, A=A, B=B, **Z)
        for stage in self.stages:
            X = stage(X, **Z)
        
        X, Z = self.dual_path(X, **Z)
        X=self.activations['gau3'](X)

        return Y, Z

@gau_test
def test1(a=1,device=None):
    print('test1')

    
@gau_test
def test2(device=None):
    print('test2')

'''
children = ['TokenScoringGAU', 'DualPathGAU', 'LatentAttentionGAU','CompressionStageGAU'] # solve redundancy 

In [28]:
code='''
# gau.py   # DO NOT CHANGE OR REMOVE THE MAKK HERE, KEEP IT ALWAYS THE FIRST LINE #

import torch
import torch.nn as nn
import torch.nn.functional as F

from model_discovery.model.utils.modules import GAUBase, gau_test # DO NOT CHANGE THIS IMPORT STATEMENT #


# YOU CAN IMPORT MORE MODULES HERE #

# YOU CAN DEFINE MORE CLASSES OR FUNCTIONS HERE #


class RelevanceScoringGAU(GAUBase):
    """Generalized Autoregressive Block Unit
        Input:        X: (batch, seqlen, embed_dim), Z: {dict of all intermediate variables} 
        Output:       Y: (batch, seqlen, embed_dim), Z_: Optional, {dict of *new* intermediate variables to update the current Z}
        Constraints:  Causal, differentiable, parameter number, complexity, parallelizable...

        embed_dim:    The dimension of the input embeddings
        block_loc:    The location of the block to be composed within the network, (layer_idx, n_block), e.g. (0, 6) for the first block in a network with 6 blocks in total
        kwarg_all:    A dictionary of all hyperparameters across all units, use it instead of kwargs to initialize the children units
    """
    def __init__(self, embed_dim: int, block_loc: tuple, kwarg_all: dict, device=None, dtype=None,**kwargs): # YOU CAN ADD MORE ARGUMENTS WITH OPTIONAL DEFAULT VALUES, BUT KEEP THE ORIGINAL ONES #
        self.factory_kwargs = {"device": device, "dtype": dtype} # DO NOT CHANGE THIS LINE, REMEMBER TO PASS IT #
        super().__init__(embed_dim, block_loc, kwarg_all) # DO NOT CHANGE THIS LINE #

        # Initialize a linear projection layer
        self.projection = nn.Linear(embed_dim, embed_dim // 2, **self.factory_kwargs)
        self.projection.weight.requires_grad = True
        self.projection.bias.requires_grad = True

        # Initialize a learnable query vector
        self.query_vector = nn.Parameter(torch.randn(embed_dim // 2, requires_grad=True, **self.factory_kwargs))

    def _forward(self, X, **Z): 
        # Project the input embeddings
        projected_X = self.projection(X)

        # Compute similarity scores
        similarity_scores = torch.matmul(projected_X, self.query_vector)

        # Normalize the similarity scores
        relevance_scores = F.softmax(similarity_scores, dim=-1)

        # Weight the input embeddings with relevance scores
        Y = X * relevance_scores.unsqueeze(-1)

        # Return weighted embeddings and updated Z
        return Y, {}


# WRITE YOUR UNIT TEST FUNCTIONS HERE #

@gau_test # DO NOT CHANGE THIS DECORATOR, OTHERWISE IT WON'T BE RECOGNIZED AS A UNIT TEST #
def test_relevance_scoring_gau(device=None, dtype=None)->None: # RENAME THIS FUNCTION, DO NOT CHANGE THE ARGUMENTS, IT SHOULD ALSO NOT RETURN ANYTHING #
    # AN AVAILABLE DEVICE AND DTYPE ARE PASSED AS ARGUMENTS, USE THEM TO INITIALIZE YOUR GAU AND MOCK INPUTS #

    # Initialize the GAU
    embed_dim = 512
    block_loc = (0, 6)
    relevance_scoring_gau = RelevanceScoringGAU(embed_dim=embed_dim, block_loc=block_loc, kwarg_all={}, device=device, dtype=dtype)

    # Mock input
    batch_size = 2
    seq_len = 10
    X = torch.randn(batch_size, seq_len, embed_dim, device=device, dtype=dtype)
    Z = {}

    # Forward pass
    Y, Z_ = relevance_scoring_gau(X, **Z)

    # Assertions
    assert Y.shape == (batch_size, seq_len, embed_dim), "Output shape mismatch"
    assert isinstance(Z_, dict), "Z_ should be a dictionary"

    # Print for debugging
    print("Output embeddings:", Y)
    print("Updated Z:", Z_)

    # Additional checks
    assert torch.allclose(Y.sum(dim=-1), X.sum(dim=-1), atol=1e-6), "Weighted embeddings should preserve the sum along the embedding dimension"

    print("RelevanceScoringGAU test passed!")
'''
children=['ProjectionGAU','SimilarityComputationGAU','NormalizationGAU']

In [43]:
code='''
# gau.py   # DO NOT CHANGE OR REMOVE THE MAKK HERE, KEEP IT ALWAYS THE FIRST LINE #

import torch
import torch.nn as nn
import torch.nn.functional as F
from model_discovery.model.utils.modules import GAUBase, gau_test # DO NOT CHANGE THIS IMPORT STATEMENT #

# YOU CAN IMPORT MORE MODULES HERE #

# YOU CAN DEFINE MORE CLASSES OR FUNCTIONS HERE #


class EnhancedGatedMLP(GAUBase):
    """Generalized Autoregressive Block Unit
        Input:        X: (batch, seqlen, embed_dim), Z: {dict of all intermediate variables} 
        Output:       Y: (batch, seqlen, embed_dim), Z_: Optional, {dict of *new* intermediate variables to update the current Z}
        Constraints:  Causal, differentiable, parameter number, complexity, parallelizable...

        embed_dim:    The dimension of the input embeddings
        block_loc:    The location of the block to be composed within the network, (layer_idx, n_block), e.g. (0, 6) for the first block in a network with 6 blocks in total
        kwarg_all:    A dictionary of all hyperparameters across all units, use it instead of kwargs to initialize the children units
    """
    def __init__(self, embed_dim: int, block_loc: tuple, kwarg_all: dict, device=None, dtype=None, hidden_features=None, out_features=None, activation=None, bias=False, multiple_of=128, **kwargs): # YOU CAN ADD MORE ARGUMENTS WITH OPTIONAL DEFAULT VALUES, BUT KEEP THE ORIGINAL ONES #
        self.factory_kwargs = {"device": device, "dtype": dtype} # DO NOT CHANGE THIS LINE, REMEMBER TO PASS IT #
        super().__init__(embed_dim, block_loc, kwarg_all) # DO NOT CHANGE THIS LINE #

        out_features = out_features if out_features is not None else embed_dim
        hidden_features = (hidden_features if hidden_features is not None else int(8 * embed_dim / 3))
        hidden_features = (hidden_features + multiple_of - 1) // multiple_of * multiple_of

        self.fc1 = nn.Linear(embed_dim, 2 * hidden_features, bias=bias, **self.factory_kwargs)
        self.activation = activation if activation is not None else F.silu
        self.fc2 = nn.Linear(hidden_features, out_features, bias=bias, **self.factory_kwargs)

        # Feedback loop components
        self.feedback_fc = nn.Linear(out_features, hidden_features, bias=bias, **self.factory_kwargs)
        self.memory_buffer = nn.Parameter(torch.zeros(hidden_features, **self.factory_kwargs))
        self.layer_norm = nn.LayerNorm(hidden_features, **self.factory_kwargs)

    def _forward(self, X, **Z): 
        # Forward pass through the first linear layer
        y = self.fc1(X)
        y, gate = y.chunk(2, dim=-1)
        y = y * self.activation(gate)

        # Feedback loop
        feedback = self.feedback_fc(y)
        feedback = self.layer_norm(feedback + self.memory_buffer)

        # Update memory buffer
        self.memory_buffer.data = feedback.detach()

        # Forward pass through the second linear layer
        y = self.fc2(feedback)
        return y, {}


# WRITE YOUR UNIT TEST FUNCTIONS HERE #

@gau_test # DO NOT CHANGE THIS DECORATOR, OTHERWISE IT WON'T BE RECOGNIZED AS A UNIT TEST #
def test_enhanced_gated_mlp(device=None, dtype=None)->None: # RENAME THIS FUNCTION, DO NOT CHANGE THE ARGUMENTS, IT SHOULD ALSO NOT RETURN ANYTHING #
    # AN AVAILABLE DEVICE AND DTYPE ARE PASSED AS ARGUMENTS, USE THEM TO INITIALIZE YOUR GAU AND MOCK INPUTS #

    embed_dim = 64
    block_loc = (0, 1)
    kwarg_all = {}
    model = EnhancedGatedMLP(embed_dim=embed_dim, block_loc=block_loc, kwarg_all=kwarg_all, device=device, dtype=dtype)

    X = torch.randn(2, 10, embed_dim, device=device, dtype=dtype)
    Y, Z = model(X)

    assert Y.shape == X.shape, f"Expected output shape {X.shape}, but got {Y.shape}"
    assert isinstance(Z, dict), "Expected Z to be a dictionary"

    print("EnhancedGatedMLP test passed!")

    # WRITE ASSERTIONS TO PERFORM THE TEST, USE PRINT TO DEBUG #

    # raise NotImplementedError # YOU MUST IMPLEMENT THIS FUNCTION #
'''

children=[]


In [45]:
unit_name = "MHA"  # Provide the unit_name to rename GAU class
reformatted_code, new_args, gau_tests, errors, warnings, fetal_errors = check_and_reformat_gau_code(code, unit_name,children)
print("Reformatted Code:\n" + reformatted_code)
print("Errors:\n", '\n\t'.join(errors+fetal_errors))
print("Warnings:\n", '\n\t'.join(warnings))
print("New Arguments:\n", new_args)


ckpt_dir = os.environ['CKPT_DIR']
lib_dir = U.pjoin(ckpt_dir, 'test_composer', 'lib')
test_tree = GAUTree('TestTree', None, None, None,None,lib_dir)
spec=P.UnitSpec(
   unitname = unit_name,
   document = 'Test',
   inputs = [],
   outputs = [],
)
test_tree.add_unit(
    spec, reformatted_code, new_args, None, None, None, children, gau_tests, None
)


Reformatted Code:
import torch
import torch.nn as nn
from model_discovery.model.utils.modules import GAUBase, gau_test
import torch.nn.functional as F


class MHA(GAUBase):
    """Generalized Autoregressive Block Unit
        Input:        X: (batch, seqlen, embed_dim), Z: {dict of all intermediate variables} 
        Output:       Y: (batch, seqlen, embed_dim), Z_: Optional, {dict of *new* intermediate variables to update the current Z}
        Constraints:  Causal, differentiable, parameter number, complexity, parallelizable...

        embed_dim:    The dimension of the input embeddings
        block_loc:    The location of the block to be composed within the network, (layer_idx, n_block), e.g. (0, 6) for the first block in a network with 6 blocks in total
        kwarg_all:    A dictionary of all hyperparameters across all units, use it instead of kwargs to initialize the children units
    """

    def __init__(self, embed_dim: int, block_loc: tuple, kwarg_all: dict,
        devic

In [39]:
x=[{'role': 'system', 'content': '\nYou are a professional AI researcher focusing on discovering the best\nautoregressive language model block. Your goal is to design a novel block\nfollowing the Generalized Autoregressive Block (GAB) structure defined in the\nfollowing base class:\n\n```python class GABBase(nn.Module):\n """ Base class for Generalized Autoregressive Block """\n def __init__(self,embed_dim: int, block_loc: tuple):\n super().__init__()\n self.embed_dim = embed_dim\n self.block_loc = block_loc # location of a block within the network, (layer_idx, n_block)\n\n def _forward(self, X, **Z): \n raise NotImplementedError\n \n # YOU ARE NOT ALLOW TO OVERRIDE THIS METHOD #\n def forward(self, X, **Z): # kwargs not parsable by torchscript but more flexible\n """Forward pass of the model"""\n assert len(X.shape) == 3, f"Input shape must be (batch, seqlen, embed_dim), got {len(X.shape)}D instead"\n assert X.shape[-1] == self.embed_dim, f"Input shape must be (batch, seqlen, embed_dim), got {X.shape} instead"\n Y = self._forward(X, **Z)\n if isinstance(Y, tuple):\n Y, Z_ = Y\n else:\n Z_ = {}\n assert Y.shape == X.shape, f"GAB Output shape must be the same as input shape of {X.shape}, got {Y.shape} instead"\n assert isinstance(Z, dict), "Intermediate variables must be stored in a dict"\n Z.update(Z_) # the new intermediate variables are updated to the current Z\n return Y, Z\n ```\n\nThe GAB will be used to construct a Generalized Autoregressive Model (GAM)\ndefined as follows:\n\n```python from transformers.modeling_outputs import CausalLMOutput\nfrom transformers import PreTrainedModel, PretrainedConfig\nfrom dataclasses import dataclass\n\nimport torch\nfrom torch import nn\n\nfrom gab import GAB, gab_config\n\n\n@dataclass\nclass GAMConfig(PretrainedConfig):\n \'\'\'Configurations for Generalized Autoregressive Models.\'\'\'\n\n d_model: int\n n_block: int\n batch_tokens: int \n vocab_size: int = None\n\n\nclass GAM(nn.Module):\n \'\'\' Generalized Autoregressive Models\n Input: X: (batch, seqlen, embed_dim)\n Output: Y: (batch, seqlen, embed_dim)\n \'\'\'\n def __init__(\n self,\n d_model: int,\n n_block: int,\n vocab_size: int = 50277,\n norm_epsilon: float = 1e-5,\n device = None,\n dtype = None,\n ) -> None:\n self.factory_kwargs = {"device": device, "dtype": dtype}\n super().__init__()\n self.d_model = d_model\n self.embedding = nn.Embedding(vocab_size, d_model, **self.factory_kwargs)\n\n block_config = gab_config()\n self.blocks = nn.ModuleList(\n [\n GAB(\n embed_dim=d_model, \n block_loc=(layer_idx,n_block),\n device=device, \n dtype=dtype, \n **block_config\n )\n for layer_idx in range(n_block)\n ]\n )\n self.norm_out = nn.LayerNorm(\n d_model, eps=norm_epsilon, **self.factory_kwargs\n )\n\n def forward(self, input_ids):\n hidden_states = self.embedding(input_ids)\n intermediate_vars = {}\n for block in self.blocks:\n hidden_states, intermediate_vars = block(\n hidden_states,\n **intermediate_vars\n )\n hidden_states = self.norm_out(hidden_states)\n return hidden_states\n\n\nclass GLMHeadModel(PreTrainedModel):\n \'\'\' Generalized Autoregressive Models with LM Head \'\'\'\n config_class = GAMConfig\n\n def __init__(\n self,\n config: GAMConfig,\n device=None,\n dtype=None,\n ) -> None:\n super().__init__(config)\n factory_kwargs = {"device": device, "dtype": dtype}\n self.backbone = GAM(\n d_model=config.d_model,\n n_block=config.n_block,\n vocab_size=config.vocab_size,\n **factory_kwargs,\n )\n self.lm_head = nn.Linear(config.d_model, config.vocab_size, bias=False, **factory_kwargs)\n\n def forward(self, input_ids, **gab_kwargs):\n hidden_states = self.backbone(input_ids, **gab_kwargs)\n lm_logits = self.lm_head(hidden_states)\n return CausalLMOutput(logits=lm_logits)\n ```\n\nThe produced language model will be pretrained with the corpus and then be\napplied for downstream tasks. The new model is expected to have a low\nperplexity, high accuracy, robustness, efficiency, and most importantly, good\nscalability. \n\nSince the autoregressive model design is complicated, so we will break it down\ninto smaller parts. We represent a block as multiple nested units, the\nGeneralized Autoregressive Unit (GAU). Each GAU accepts a sequence of embeddings\nX and a dictionary of intermediate variables Z as input, and outputs a sequence\nof embeddings Y and a dictionary of new or updated intermediate variables Z_. Z_\nis optional, when it is provided, it will be used to update Z for the next unit\nby Z.update(Z_). A GAU is defined in the following base class:\n\n```python class GAUBase(nn.Module): \n """ \n Instead of directly giving the full implementation of a GAB block, the agent need to \n design a series of nested GAB units and construct the full GAB block as a pipeline of these units.\n\n GAB is fractal, like GAB itself, each GAB unit accepts X and Z as input and returns Y and Z as output.\n """ \n def __init__(self, embed_dim: int, block_loc: tuple, kwarg_all: dict):\n super().__init__()\n self.embed_dim = embed_dim\n self.block_loc = block_loc # location of a block within the network, (layer_idx, n_block)\n self.kwarg_all = kwarg_all # dict of all kwargs, use it to initialize the children units\n\n def _forward(self, X, **Z): # you can define the arguments instead of using **Z\n raise NotImplementedError\n \n def forward(self, X, **Z):\n assert len(X.shape) == 3 and X.shape[-1] == self.embed_dim, f"Input X must be a sequence of shape (batch, seqlen, embed_dim), all other kinds of variables should be passed by Z. Got {X.shape} instead."\n _params = inspect.signature(self._forward).parameters\n X=X.to(**self.factory_kwargs)\n _Z = {k: v for k, v in Z.items() if k in _params}\n Y = self._forward(X, **_Z)\n if isinstance(Y, tuple):\n Y, Z_ = Y\n else:\n Z_ = {}\n assert Y.shape == X.shape, f"GAU output Y must be a sequence with the same shape as input of {X.shape}, got {Y.shape} instead, all other kinds of variables should be passed by Z."\n assert isinstance(Z_, dict), "Intermediate variables must be stored in a dict"\n Z.update(Z_) # the new intermediate variables are updated to the current Z\n return Y, Z\n ```\n\nYou will design a GAU by completing the blanks marked in this template, which\nincludes the initialization where you can define your custom arguments with\noptional default values, the forward function where you can define convenient\nfunctions or classes in the GAB class such as caches, notice that you are only\nallowed to have only one GAU which inherited from the GAUBase class in the file:\n \n```python # gau.py # DO NOT CHANGE OR REMOVE THE MAKK HERE, KEEP IT ALWAYS THE FIRST LINE #\n\nimport torch\nimport torch.nn as nn\n\nfrom model_discovery.model.utils.modules import GAUBase, gau_test # DO NOT CHANGE THIS IMPORT STATEMENT #\n\n\n# YOU CAN IMPORT MORE MODULES HERE #\n\n# YOU CAN DEFINE MORE CLASSES OR FUNCTIONS HERE #\n\n\nclass UnitName(GAUBase):\n """Generalized Autoregressive Block Unit\n Input: X: (batch, seqlen, embed_dim), Z: {dict of all intermediate variables} \n Output: Y: (batch, seqlen, embed_dim), Z_: Optional, {dict of *new* intermediate variables to update the current Z}\n Constraints: Causal, differentiable, parameter number, complexity, parallelizable...\n\n embed_dim: The dimension of the input embeddings\n block_loc: The location of the block to be composed within the network, (layer_idx, n_block), e.g. (0, 6) for the first block in a network with 6 blocks in total\n kwarg_all: A dictionary of all hyperparameters across all units, use it instead of kwargs to initialize the children units\n """\n def __init__(self, embed_dim: int, block_loc: tuple, kwarg_all: dict, device=None, dtype=None,**kwargs): # YOU CAN ADD MORE ARGUMENTS WITH OPTIONAL DEFAULT VALUES, BUT KEEP THE ORIGINAL ONES #\n self.factory_kwargs = {"device": device, "dtype": dtype} # DO NOT CHANGE THIS LINE, REMEMBER TO PASS IT #\n super().__init__(embed_dim, block_loc, kwarg_all) # DO NOT CHANGE THIS LINE #\n \n # COMPLETING THE CODE HERE #\n\n raise NotImplementedError\n\n\n # YOU CAN ADD MORE FUNCTIONS HERE #\n\n\n def _forward(self, X, **Z): \n \n # THE CODE HERE MUST BE COMPLETED #\n\n raise NotImplementedError\n\n\n# WRITE YOUR UNIT TEST FUNCTIONS HERE #\n\n@gau_test # DO NOT CHANGE THIS DECORATOR, OTHERWISE IT WON\'T BE RECOGNIZED AS A UNIT TEST #\ndef unit_test_name(device=None, dtype=None)->None: # RENAME THIS FUNCTION, DO NOT CHANGE THE ARGUMENTS, IT SHOULD ALSO NOT RETURN ANYTHING #\n # AN AVAILABLE DEVICE AND DTYPE ARE PASSED AS ARGUMENTS, USE THEM TO INITIALIZE YOUR GAU AND MOCK INPUTS #\n\n # WRITE ASSERTIONS TO PERFORM THE TEST, USE PRINT TO DEBUG #\n \n raise NotImplementedError # YOU MUST IMPLEMENT THIS FUNCTION #\n ```\n\nIn a GAU, you can call other GAUs, as such, you can create a complicated GAB\nblock by nesting multiple GAUs. However, each GAU should be not too complex, if\nyou want to create complex block, you should break it down into smaller GAUs and\nnest them. As such, you should design a GAB block in a top-down manner. \n\nInstead of starting from scratch, you will start from an existing design and\nimprove it. You will be provided with the full information of the design,\nincluding the proposal, the tree structure, the implementations of the GAUs. You\nare only allowed to modify *one GAU* from the existing design. \n\nYou need to select one GAU to modify, you can define new children GAUs, however\nyou need to guarantee that your modification wont affect the correctness of the\noverall design. \n\nYou will start by writing down an overal proposal for the design you want to\nhave, the proposal decides a direction, phylosophy and the plan of the design,\nand the analysis of the problem and how you gonna solve it by modifying one GAU\nfrom the existing design. You will be provided with one or multiple references\nto consider that may inspire you if there are references provided.\n\nYour response should include: \n\n1. The proposal, it should include but not restrict to the following parts: a. A\n title with the name of the design in the level 1 header format. You shuld\n have only one level 1 header in your response which is the name of the\n design.\n\n b. Your motivation of the design. What problem you want to solve based on the\n insights or observations you have about the autoregressive models today,\n and any inspirations you may have from the references. \n\n c. The analysis of the problem.\n\n d. The core idea and phylosophy behind of your design that may solve the\n problem you proposed. \n\n e. The plan of the design. You should include subsections of that describe\n the details of each part of the design with the justifications. The\n selection of the GAU to modify and the reasoning of the selection. \n\n f. A conclution of the proposal. \n\n g. Optional, the references you used in your proposal, should be in the right\n format.\n2. The name of the variant of the model you are going to design.\n3. The selection of the GAU to modify.\n\nThe proposal will be reviewed and you will be asked to modify it if it is not\npassed. You can start to implement the design after the proposal is passed. \n\nThe proposal should be as detailed as possible, DO NOT WORRY IF THE PROPOSAL IS\nTOO LONG, BUT ALSO DO NOT FILL IN BY REDUNDANT WORDS, USE PRECISE AND CONCRETE\nLANGUAGE, the proposal will be the guideline for the entire design process so it\nshould be clear and detailed. \n'}]
print(x[0]['content'])


You are a professional AI researcher focusing on discovering the best
autoregressive language model block. Your goal is to design a novel block
following the Generalized Autoregressive Block (GAB) structure defined in the
following base class:

```python class GABBase(nn.Module):
 """ Base class for Generalized Autoregressive Block """
 def __init__(self,embed_dim: int, block_loc: tuple):
 super().__init__()
 self.embed_dim = embed_dim
 self.block_loc = block_loc # location of a block within the network, (layer_idx, n_block)

 def _forward(self, X, **Z): 
 raise NotImplementedError
 
 # YOU ARE NOT ALLOW TO OVERRIDE THIS METHOD #
 def forward(self, X, **Z): # kwargs not parsable by torchscript but more flexible
 """Forward pass of the model"""
 assert len(X.shape) == 3, f"Input shape must be (batch, seqlen, embed_dim), got {len(X.shape)}D instead"
 assert X.shape[-1] == self.embed_dim, f"Input shape must be (batch, seqlen, embed_dim), got {X.shape} instead"
 Y = self._forward(X, **Z)


In [30]:
code=test_tree.compose_unit('XAU')
code=U.add_line_num(code)
print(code)

line 1: import torch
line 2: import torch.nn as nn
line 3: from model_discovery.model.utils.modules import GAUBase, gau_test
line 4: import torch.nn.functional as F
line 5: 
line 6: 
line 7: class XAU(GAUBase):
line 8:     """Generalized Autoregressive Block Unit
line 9:         Input:        X: (batch, seqlen, embed_dim), Z: {dict of all intermediate variables} 
line 10:         Output:       Y: (batch, seqlen, embed_dim), Z_: Optional, {dict of *new* intermediate variables to update the current Z}
line 11:         Constraints:  Causal, differentiable, parameter number, complexity, parallelizable...
line 12: 
line 13:         embed_dim:    The dimension of the input embeddings
line 14:         block_loc:    The location of the block to be composed within the network, (layer_idx, n_block), e.g. (0, 6) for the first block in a network with 6 blocks in total
line 15:         kwarg_all:    A dictionary of all hyperparameters across all units, use it instead of kwargs to initialize the chi

In [31]:
code='''
import torch
import torch.nn as nn
from model_discovery.model.utils.modules import GAUBase, gau_test


class DilatedReformerRootGAU(GAUBase):
    """Generalized Autoregressive Block Unit
        Input:        X: (batch, seqlen, embed_dim), Z: {dict of all intermediate variables} 
        Output:       Y: (batch, seqlen, embed_dim), Z_: Optional, {dict of *new* intermediate variables to update the current Z}
        Constraints:  Causal, differentiable, parameter number, complexity, parallelizable...

        embed_dim:    The dimension of the input embeddings
        block_loc:    The location of the block to be composed within the network, (layer_idx, n_block), e.g. (0, 6) for the first block in a network with 6 blocks in total
        kwarg_all:    A dictionary of all hyperparameters across all units, use it instead of kwargs to initialize the children units
    """

    def __init__(self, embed_dim: int, block_loc: tuple, kwarg_all: dict,
        device=None, dtype=None, **kwargs):
        self.factory_kwargs = {'device': device, 'dtype': dtype}
        super().__init__(embed_dim, block_loc, kwarg_all)
        self.dilated_attention = DilatedAttentionGAU(embed_dim=embed_dim,
            block_loc=block_loc, kwarg_all=kwarg_all, **self.factory_kwargs,
            **kwarg_all)
        self.reversible_residual = ReversibleResidualGAU(embed_dim=
            embed_dim, block_loc=block_loc, kwarg_all=kwarg_all, **
            self.factory_kwargs, **kwarg_all)
        self.lsh_attention = LSHAttentionGAU(embed_dim=embed_dim, block_loc
            =block_loc, kwarg_all=kwarg_all, **self.factory_kwargs, **kwarg_all
            )

    def _forward(self, X, **Z):
        X, Z = self.dilated_attention(X, **Z)
        X, Z = self.reversible_residual(X, **Z)
        X, Z = self.lsh_attention(X, **Z)
        return X, Z


class LSHAttentionGAU(GAUBase): 
    def __init__(self, embed_dim: int, block_loc: tuple, kwarg_all: dict, device=None, dtype=None, **kwargs): 
        self.factory_kwargs = {"device": device, "dtype": dtype} 
        super().__init__(embed_dim, block_loc, kwarg_all)

    def _forward(self, X, **Z): 
        return X


class DilatedAttentionGAU(GAUBase): 
    def __init__(self, embed_dim: int, block_loc: tuple, kwarg_all: dict, device=None, dtype=None, **kwargs): 
        self.factory_kwargs = {"device": device, "dtype": dtype} 
        super().__init__(embed_dim, block_loc, kwarg_all)

    def _forward(self, X, **Z): 
        return X


class ReversibleResidualGAU(GAUBase): 
    def __init__(self, embed_dim: int, block_loc: tuple, kwarg_all: dict, device=None, dtype=None, **kwargs): 
        self.factory_kwargs = {"device": device, "dtype": dtype} 
        super().__init__(embed_dim, block_loc, kwarg_all)

    def _forward(self, X, **Z): 
        return X


@gau_test
def test_DilatedReformerRootGAU_unit_test_dilated_reformer_root_gau(device=
    None, dtype=None) ->None:
    batch_size = 2
    seq_len = 10
    embed_dim = 16
    X = torch.randn(batch_size, seq_len, embed_dim, device=device, dtype=dtype)
    Z = {}
    gau = DilatedReformerRootGAU(embed_dim=embed_dim, block_loc=(0, 6), kwarg_all={}, device=
        device, dtype=dtype)
    Y, Z_ = gau(X, **Z)
    assert Y.shape == X.shape, 'Output shape should match input shape'
    assert isinstance(Z_, dict), 'Z_ should be a dictionary'
    print('Input X:', X)
    print('Output Y:', Y)
    print('Intermediate variables Z_:', Z_)


def run_DilatedReformerRootGAU_tests():
	try:
		test_DilatedReformerRootGAU_unit_test_dilated_reformer_root_gau()
	except Exception as e:
		print("Error in running unit_test_dilated_reformer_root_gau:", e)


if __name__ == "__main__":
	run_DilatedReformerRootGAU_tests()
'''

import io
from contextlib import redirect_stdout

output=io.StringIO()
with redirect_stdout(output):
    exec(code)
captured = str(output.getvalue())

## Debug area

In [32]:
import torch
import torch.nn as nn
from model_discovery.model.utils.modules import GABBase


class GAB(GABBase):

    def __init__(self, embed_dim: int, block_loc: tuple, device=None, dtype
        =None, **kwargs):
        factory_kwargs = {'device': device, 'dtype': dtype}
        super().__init__(embed_dim, block_loc)
        self.root = ResonanceLOCOSTGAMRoot(embed_dim=embed_dim, block_loc=
            block_loc, kwarg_all=kwargs, **factory_kwargs, **kwargs)

    def _forward(self, X, **Z):
        X, Z = self.root(X, **Z)
        return X, Z


from model_discovery.model.utils.modules import GAUBase, gau_test


class ResonanceLOCOSTGAMRoot(GAUBase):
    """Generalized Autoregressive Block Unit
        Input:        X: (batch, seqlen, embed_dim), Z: {dict of all intermediate variables} 
        Output:       Y: (batch, seqlen, embed_dim), Z_: Optional, {dict of *new* intermediate variables to update the current Z}
        Constraints:  Causal, differentiable, parameter number, complexity, parallelizable...

        embed_dim:    The dimension of the input embeddings
        block_loc:    The location of the block to be composed within the network, (layer_idx, n_block), e.g. (0, 6) for the first block in a network with 6 blocks in total
        kwarg_all:    A dictionary of all hyperparameters across all units, use it instead of kwargs to initialize the children units
    """

    def __init__(self, embed_dim: int, block_loc: tuple, kwarg_all: dict,
        device=None, dtype=None, **kwargs):
        self.factory_kwargs = {'device': device, 'dtype': dtype}
        super().__init__(embed_dim, block_loc, kwarg_all)
        self.resonance_rope = ResonanceRoPEUnit(embed_dim=embed_dim,
            block_loc=block_loc, kwarg_all=kwarg_all, **self.factory_kwargs,
            **kwarg_all)
        self.state_space = StateSpaceUnit(embed_dim=embed_dim, block_loc=
            block_loc, kwarg_all=kwarg_all, **self.factory_kwargs, **kwarg_all)
        self.inter_block_comm = InterBlockCommUnit(embed_dim=embed_dim,
            block_loc=block_loc, kwarg_all=kwarg_all, **self.factory_kwargs,
            **kwarg_all)

    def _forward(self, X, **Z):
        Y, Z_ = self.resonance_rope(X, **Z)
        Y, Z_ = self.state_space(Y, **Z_)
        Y, Z_ = self.inter_block_comm(Y, **Z_)
        return Y, Z_


import math


class ResonanceRoPEUnit(GAUBase):
    """Generalized Autoregressive Block Unit
        Input:        X: (batch, seqlen, embed_dim), Z: {dict of all intermediate variables} 
        Output:       Y: (batch, seqlen, embed_dim), Z_: Optional, {dict of *new* intermediate variables to update the current Z}
        Constraints:  Causal, differentiable, parameter number, complexity, parallelizable...

        embed_dim:    The dimension of the input embeddings
        block_loc:    The location of the block to be composed within the network, (layer_idx, n_block), e.g. (0, 6) for the first block in a network with 6 blocks in total
        kwarg_all:    A dictionary of all hyperparameters across all units, use it instead of kwargs to initialize the children units
    """

    def __init__(self, embed_dim: int, block_loc: tuple, kwarg_all: dict,
        device=None, dtype=None, **kwargs):
        self.factory_kwargs = {'device': device, 'dtype': dtype}
        super().__init__(embed_dim, block_loc, kwarg_all)
        self.freqs = self._init_frequencies(embed_dim)

    def _init_frequencies(self, embed_dim):
        half_dim = embed_dim // 2
        freqs = torch.exp(-torch.arange(half_dim, dtype=torch.float32,
            device=self.factory_kwargs['device']) * (math.log(10000.0) / (
            half_dim - 1)))
        return freqs

    def _apply_rope(self, X):
        batch_size, seqlen, _ = X.size()
        position_ids = torch.arange(seqlen, dtype=X.dtype, device=X.device)
        sinusoid_inp = torch.einsum('i,j->ij', position_ids, self.freqs)
        sin, cos = sinusoid_inp.sin(), sinusoid_inp.cos()
        X1, X2 = X[..., ::2], X[..., 1::2]
        Y1 = X1 * cos - X2 * sin
        Y2 = X1 * sin + X2 * cos
        return torch.stack([Y1, Y2], dim=-1).flatten(-2)

    def _forward(self, X, **Z):
        Y = self._apply_rope(X)
        Z_ = Z
        return Y, Z_


class StateSpaceUnit(GAUBase):

    def __init__(self, embed_dim: int, block_loc: tuple, kwarg_all: dict,
        device=None, dtype=None, **kwargs):
        self.factory_kwargs = {'device': device, 'dtype': dtype}
        super().__init__(embed_dim, block_loc, kwarg_all)

    def _forward(self, X, **Z):
        return X


class InterBlockCommUnit(GAUBase):

    def __init__(self, embed_dim: int, block_loc: tuple, kwarg_all: dict,
        device=None, dtype=None, **kwargs):
        self.factory_kwargs = {'device': device, 'dtype': dtype}
        super().__init__(embed_dim, block_loc, kwarg_all)

    def _forward(self, X, **Z):
        return X
    


gab_config = {}


# Check if to device still work!

gab = GAB(128, (0, 6))

In [33]:
def get_module_device(model):
    """
    Returns the device of the first parameter found in the given nn.Module.
    If the module has no parameters, returns None.
    """
    return next(model.parameters()).device

get_module_device(gab.root)


StopIteration: 

In [ ]:
next(gab.parameters())

StopIteration: 

## Tree

In [ ]:
children={
    'unit1': 'MockUnit1',
    'unit2': 'MockUnit2',
}
called=['unit1', 'unit2', 'unit1']
test_tree.units['DualPathGAU'] = GAUNode('DualPathGAU', None, None, None, called, None, None, children, None)
children={
    'unit1': 'MockUnit3',
    'unit2': 'MockUnit4',
}
called=['unit2', 'unit2']
test_tree.units['MockUnit1'] = GAUNode('MockUnit1', None, None, None, called, None, None, children, None)

In [ ]:
test_tree.units

{'DualPathGAU': GAUNode(name='DualPathGAU', code=None, args=None, desc=None, path=['unit1', 'unit2', 'unit1'], review=None, rating=None, children={'unit1': 'MockUnit1', 'unit2': 'MockUnit2'}, suggestions=None),
 'MockUnit1': GAUNode(name='MockUnit1', code=None, args=None, desc=None, path=['unit2', 'unit2'], review=None, rating=None, children={'unit1': 'MockUnit3', 'unit2': 'MockUnit4'}, suggestions=None)}

In [ ]:
class A:
    def __init__(self,r,d,e,**Z):
        print('Args in A:',d,e)
        print('Kwargs in A:',Z)

class B(A):
    def __init__(self,_Z,c=1,d=2,e=3,**Z):
        args2={'d':d,'e':e}
        super().__init__(c,**_Z,**args2)
        print('Kwargs in B:',Z)

cfg={
    'a':1,
    'b':2,
    'c':3,
}

B(cfg,**cfg)

Args in A: 2 3
Kwargs in A: {'a': 1, 'b': 2, 'c': 3}
Kwargs in B: {'a': 1, 'b': 2}


In [ ]:
def A(X,Z,**kwargs):
    print(X)
    print(Z)
    print(kwargs)

def B(X,Z,a,**kwargs):
    A(X,Z,**Z)

Z={'a':3}
B(1,Z,**Z)

1
{'a': 3}
{'a': 3}


## Test


In [ ]:
import os,sys
sys.path.append('..')
import torch
import torch.nn as nn
from model_discovery.model.utils.modules import GABBase


class GAB(GABBase):
    '''
    GAU Tree Map of Butterfly-AFT Generalized Autoregressive Block (BAGAB):
    ButterflyAFTGAU (Rating: 3.5/5)
        |- AFTMechanismGAU (Rating: 3.5/5)
            |- PositionBiasGAU (Rating: 3.5/5)
            |- ElementWiseOperationGAU (Rating: 3.5/5)
        |- ButterflyTransformGAU (Rating: 3.5/5)
            |- ButterflyLayerGAU (Rating: 3.5/5)
                |- ButterflyStageGAU (Rating: 3.5/5)
                    |- ButterflyParameterGAU (Unimplemented)
            |- ButterflyInitializationGAU (Rating: 3.5/5)
                |- ButterflyStageGAU (Rating: 3.5/5)
                    |- ButterflyParameterGAU (Unimplemented)
                |- ButterflyParameterGAU (Unimplemented)

    Implemented Units: AFTMechanismGAU, ButterflyLayerGAU, PositionBiasGAU, ButterflyTransformGAU, ElementWiseOperationGAU, ButterflyAFTGAU, ButterflyInitializationGAU, ButterflyStageGAU
    Unimplemented Units: AFTMechanismGAU, ButterflyLayerGAU, PositionBiasGAU, ButterflyTransformGAU, ButterflyParameterGAU, ElementWiseOperationGAU, ButterflyInitializationGAU, ButterflyStageGAU
    '''

    def __init__(self, embed_dim: int, block_loc: tuple, device=None, dtype
        =None, **kwargs):
        factory_kwargs = {'device': device, 'dtype': dtype}
        super().__init__(embed_dim, block_loc)
        self.root = ButterflyAFTGAU(embed_dim=embed_dim, block_loc=
            block_loc, kwarg_all=kwargs, **factory_kwargs, **kwargs)

    def _forward(self, X, **Z):
        X, Z = self.root(X, **Z)
        return X, Z


from model_discovery.model.utils.modules import GAUBase


class ButterflyAFTGAU(GAUBase):
    """Generalized Autoregressive Block Unit
        Input:        X: (batch, seqlen, embed_dim), Z: {dict of all intermediate variables} 
        Output:       Y: (batch, seqlen, embed_dim), Z_: Optional, {dict of *new* intermediate variables to update the current Z}
        Constraints:  Causal, differentiable, parameter number, complexity, parallelizable...

        embed_dim:    The dimension of the input embeddings
        block_loc:    The location of the block within the network, (layer_idx, n_block)
        kwarg_all:    A dictionary of all hyperparameters across all units, use it instead of kwargs to initialize the children units
    """

    def __init__(self, embed_dim: int, block_loc: tuple, kwarg_all: dict,
        device=None, dtype=None, **kwargs):
        self.factory_kwargs = {'device': device, 'dtype': dtype}
        super().__init__(embed_dim, block_loc, kwarg_all)
        self.aft_mechanism = AFTMechanismGAU(embed_dim=embed_dim, block_loc
            =block_loc, kwarg_all=kwarg_all, **self.factory_kwargs, **kwarg_all
            )
        self.butterfly_transform = ButterflyTransformGAU(embed_dim=
            embed_dim, block_loc=block_loc, kwarg_all=kwarg_all, **self.
            factory_kwargs, **kwarg_all)

    def _forward(self, X, **Z):
        aft_output, Z = self.aft_mechanism(X, **Z)
        butterfly_output, Z = self.butterfly_transform(aft_output, **Z)
        return butterfly_output, Z


class ButterflyTransformGAU(GAUBase):

    def __init__(self, embed_dim: int, block_loc: tuple, kwarg_all: dict,
        device=None, dtype=None, **kwargs):
        self.factory_kwargs = {'device': device, 'dtype': dtype}
        super().__init__(embed_dim, block_loc, kwarg_all)
        self.butterfly_layer = ButterflyLayerGAU(embed_dim=embed_dim,
            block_loc=block_loc, kwarg_all=kwarg_all, **self.factory_kwargs,
            **kwarg_all)
        self.butterfly_init = ButterflyInitializationGAU(embed_dim=
            embed_dim, block_loc=block_loc, kwarg_all=kwarg_all, **self.
            factory_kwargs, **kwarg_all)

    def _forward(self, X, **Z):
        X, Z = self.butterfly_init(X, **Z)
        Y, Z = self.butterfly_layer(X, **Z)
        return Y, Z


class ButterflyLayerGAU(GAUBase):
    """Generalized Autoregressive Block Unit
        Input:        X: (batch, seqlen, embed_dim), Z: {dict of all intermediate variables} 
        Output:       Y: (batch, seqlen, embed_dim), Z_: Optional, {dict of *new* intermediate variables to update the current Z}
        Constraints:  Causal, differentiable, parameter number, complexity, parallelizable...

        embed_dim:    The dimension of the input embeddings
        block_loc:    The location of the block within the network, (layer_idx, n_block)
        kwarg_all:    A dictionary of all hyperparameters across all units, use it instead of kwargs to initialize the children units
    """

    def __init__(self, embed_dim: int, block_loc: tuple, kwarg_all: dict,
        device=None, dtype=None, **kwargs):
        self.factory_kwargs = {'device': device, 'dtype': dtype}
        super().__init__(embed_dim, block_loc, kwarg_all)
        self.stages = nn.ModuleList([ButterflyStageGAU(embed_dim=embed_dim,
            block_loc=block_loc, kwarg_all=kwarg_all, **self.factory_kwargs,
            **kwarg_all) for _ in range(kwarg_all.get('num_stages', 4))])

    def _forward(self, X, **Z):
        for stage in self.stages:
            X, Z = stage(X, **Z)
        return X, Z

class ButterflyMatrixGAU(GAUBase):

    def __init__(self, embed_dim: int, block_loc: tuple, kwarg_all: dict,
        device=None, dtype=None, **kwargs):
        self.factory_kwargs = {'device': device, 'dtype': dtype}
        super().__init__(embed_dim, block_loc, kwarg_all)

    def _forward(self, X, **Z):
        return X


import torch.nn.functional as F


class ButterflyInitializationGAU(GAUBase):
    """Generalized Autoregressive Block Unit
        Input:        X: (batch, seqlen, embed_dim), Z: {dict of all intermediate variables} 
        Output:       Y: (batch, seqlen, embed_dim), Z_: Optional, {dict of *new* intermediate variables to update the current Z}
        Constraints:  Causal, differentiable, parameter number, complexity, parallelizable...

        embed_dim:    The dimension of the input embeddings
        block_loc:    The location of the block within the network, (layer_idx, n_block)
        kwarg_all:    A dictionary of all hyperparameters across all units, use it instead of kwargs to initialize the children units
    """

    def __init__(self, embed_dim: int, block_loc: tuple, kwarg_all: dict,
        device=None, dtype=None, **kwargs):
        self.factory_kwargs = {'device': device, 'dtype': dtype}
        super().__init__(embed_dim, block_loc, kwarg_all)
        self.butterfly_size = kwarg_all.get('butterfly_size', embed_dim)
        self.butterfly_matrices = nn.ParameterList([nn.Parameter(torch.
            randn(self.butterfly_size, self.butterfly_size, **self.
            factory_kwargs)) for _ in range(int(torch.log2(torch.tensor(
            self.butterfly_size))))])
        self.butterfly_stage = ButterflyStageGAU(embed_dim=embed_dim,
            block_loc=block_loc, kwarg_all=kwarg_all, **self.factory_kwargs,
            **kwarg_all)
        self.butterfly_parameter = ButterflyParameterGAU(embed_dim=
            embed_dim, block_loc=block_loc, kwarg_all=kwarg_all, **self.
            factory_kwargs, **kwarg_all)

    def _forward(self, X, **Z):
        for matrix in self.butterfly_matrices:
            X = F.linear(X, matrix)
        return X, Z


class ButterflyParameterGAU(GAUBase):
    """
    Generalized Autoregressive Block Unit for learning parameters of Butterfly Factorization.
    Input:        X: (batch, seqlen, embed_dim), Z: {dict of all intermediate variables} 
    Output:       Y: (batch, seqlen, embed_dim), Z_: Optional, {dict of *new* intermediate variables to update the current Z}
    Constraints:  Causal, differentiable, parameter number, complexity, parallelizable...

    embed_dim:    The dimension of the input embeddings
    block_loc:    The location of the block within the network, (layer_idx, n_block)
    kwarg_all:    A dictionary of all hyperparameters across all units, use it instead of kwargs to initialize the children units
    """

    def __init__(self, embed_dim: int, block_loc: tuple, kwarg_all: dict,
        device=None, dtype=None, **kwargs):
        self.factory_kwargs = {'device': device, 'dtype': dtype}
        super().__init__(embed_dim, block_loc, kwarg_all)
        self.param_size = kwarg_all.get('param_size', 128)
        self.butterfly_weights = nn.Parameter(torch.randn(self.param_size,
            embed_dim, **self.factory_kwargs))
        nn.init.xavier_uniform_(self.butterfly_weights)

    def _forward(self, X, **Z):
        transformed_X = torch.matmul(X, self.butterfly_weights)
        Z_ = {'butterfly_weights': self.butterfly_weights}
        return transformed_X, Z_
        # transformed_X = F.linear(X, self.butterfly_weights.T)
        # return transformed_X, {}


class ButterflyComputationGAU(GAUBase):

    def __init__(self, embed_dim: int, block_loc: tuple, kwarg_all: dict,
        device=None, dtype=None, **kwargs):
        self.factory_kwargs = {'device': device, 'dtype': dtype}
        super().__init__(embed_dim, block_loc, kwarg_all)

    def _forward(self, X, **Z):
        return X


class ButterflyStageGAU(GAUBase):

    def __init__(self, embed_dim: int, block_loc: tuple, kwarg_all: dict,
        device=None, dtype=None, **kwargs):
        self.factory_kwargs = {'device': device, 'dtype': dtype}
        super().__init__(embed_dim, block_loc, kwarg_all)
        self.butterfly_params = ButterflyParameterGAU(embed_dim=embed_dim,
            block_loc=block_loc, kwarg_all=kwarg_all, **self.factory_kwargs,
            **kwarg_all)

    def _forward(self, X, **Z):
        transformed_X, Z = self.butterfly_params(X, **Z)
        assert transformed_X.shape == X.shape, f'Output shape {transformed_X.shape} does not match input shape {X.shape}'
        return transformed_X, Z

class AFTMechanismGAU(GAUBase):
    """Generalized Autoregressive Block Unit
        Input:        X: (batch, seqlen, embed_dim), Z: {dict of all intermediate variables} 
        Output:       Y: (batch, seqlen, embed_dim), Z_: Optional, {dict of *new* intermediate variables to update the current Z}
        Constraints:  Causal, differentiable, parameter number, complexity, parallelizable...

        embed_dim:    The dimension of the input embeddings
        block_loc:    The location of the block within the network, (layer_idx, n_block)
        kwarg_all:    A dictionary of all hyperparameters across all units, use it instead of kwargs to initialize the children units
    """

    def __init__(self, embed_dim: int, block_loc: tuple, kwarg_all: dict,
        device=None, dtype=None, **kwargs):
        self.factory_kwargs = {'device': device, 'dtype': dtype}
        super().__init__(embed_dim, block_loc, kwarg_all)
        self.position_bias = PositionBiasGAU(embed_dim=embed_dim, block_loc
            =block_loc, kwarg_all=kwarg_all, **self.factory_kwargs, **kwarg_all
            )
        self.element_wise_operation = ElementWiseOperationGAU(embed_dim=
            embed_dim, block_loc=block_loc, kwarg_all=kwarg_all, **self.
            factory_kwargs, **kwarg_all)

    def _forward(self, X, **Z):
        X, Z = self.position_bias(X, **Z)
        Y, Z = self.element_wise_operation(X, **Z)
        return Y, Z


class ElementWiseOperationGAU(GAUBase):
    """Generalized Autoregressive Block Unit
        Input:        X: (batch, seqlen, embed_dim), Z: {dict of all intermediate variables} 
        Output:       Y: (batch, seqlen, embed_dim), Z_: Optional, {dict of *new* intermediate variables to update the current Z}
        Constraints:  Causal, differentiable, parameter number, complexity, parallelizable...

        embed_dim:    The dimension of the input embeddings
        block_loc:    The location of the block within the network, (layer_idx, n_block)
        kwarg_all:    A dictionary of all hyperparameters across all units, use it instead of kwargs to initialize the children units
    """

    def __init__(self, embed_dim: int, block_loc: tuple, kwarg_all: dict,
        device=None, dtype=None, **kwargs):
        self.factory_kwargs = {'device': device, 'dtype': dtype}
        super().__init__(embed_dim, block_loc, kwarg_all)
        self.position_bias = nn.Parameter(torch.zeros(1, embed_dim, **self.
            factory_kwargs))

    def _forward(self, X, **Z):
        queries = Z.get('queries', X)
        keys = Z.get('keys', X)
        values = Z.get('values', X)
        elementwise_product = queries * keys
        elementwise_sum = elementwise_product + values + self.position_bias
        return elementwise_sum, {}


class PositionBiasGAU(GAUBase):
    """Position Bias Generalized Autoregressive Unit
        Input:        X: (batch, seqlen, embed_dim), Z: {dict of all intermediate variables} 
        Output:       Y: (batch, seqlen, embed_dim), Z_: Optional, {dict of *new* intermediate variables to update the current Z}
        Constraints:  Causal, differentiable, parameter number, complexity, parallelizable...

        embed_dim:    The dimension of the input embeddings
        block_loc:    The location of the block within the network, (layer_idx, n_block)
        kwarg_all:    A dictionary of all hyperparameters across all units, use it instead of kwargs to initialize the children units
    """

    def __init__(self, embed_dim: int, block_loc: tuple, kwarg_all: dict,
        device=None, dtype=None, **kwargs):
        self.factory_kwargs = {'device': device, 'dtype': dtype}
        super().__init__(embed_dim, block_loc, kwarg_all)
        self.position_bias = nn.Parameter(torch.zeros(1, 1, embed_dim, **
            self.factory_kwargs))

    def _forward(self, X, **Z):
        Y = X + self.position_bias
        return Y, {}
    

class GABLM(nn.Module):
    def __init__(self, embed_dim, vocab_size=100, **kwargs):
        super().__init__()
        self.gab = GAB(embed_dim, (0,1), **kwargs)
        self.lm_head = nn.Linear(embed_dim, vocab_size, bias=False)
    
    def forward(self, X, **Z):
        X, Z = self.gab(X, **Z)
        return self.lm_head(X),X



gab_config = {}


In [ ]:
class SimpleGABLM(nn.Module):
    def __init__(self, embed_dim, vocab_size=100):
        super().__init__()
        self.butterfly_param = ButterflyParameterGAU(embed_dim, (0,1), {})
        self.lm_head = nn.Linear(embed_dim, vocab_size, bias=False)
    
    def forward(self, X, **Z):
        X, _ = self.butterfly_param(X, **Z)
        return self.lm_head(X), X

# Test the simplified model
simple_model = GABLM(128)
mock_input = torch.randn(1, 10, 128)
mock_y = torch.randint(0, 100, (1, 10))

optimizer = optim.Adam(simple_model.parameters())
criterion = nn.CrossEntropyLoss()

# Training loop
for _ in range(5):  # Run for a few iterations
    optimizer.zero_grad()
    y, X = simple_model(mock_input)
    loss = criterion(y.view(-1, 100), mock_y.view(-1))
    loss.backward()
    print(f"Loss: {loss.item()}")
    for name, param in simple_model.named_parameters():
        if param.grad is not None:
            print(f"Gradient for {name}: {param.grad.abs().mean()}")
        else:
            print(f"No gradient for {name}")
    optimizer.step()

Loss: 54495796.0
Gradient for gab.root.aft_mechanism.position_bias.position_bias: 899675.4375
Gradient for gab.root.aft_mechanism.element_wise_operation.position_bias: 475023.78125
Gradient for gab.root.butterfly_transform.butterfly_layer.stages.0.butterfly_params.butterfly_weights: 810216.875
Gradient for gab.root.butterfly_transform.butterfly_layer.stages.1.butterfly_params.butterfly_weights: 777344.75
Gradient for gab.root.butterfly_transform.butterfly_layer.stages.2.butterfly_params.butterfly_weights: 783032.0625
Gradient for gab.root.butterfly_transform.butterfly_layer.stages.3.butterfly_params.butterfly_weights: 771978.3125
Gradient for gab.root.butterfly_transform.butterfly_init.butterfly_matrices.0: 62701.796875
Gradient for gab.root.butterfly_transform.butterfly_init.butterfly_matrices.1: 66148.203125
Gradient for gab.root.butterfly_transform.butterfly_init.butterfly_matrices.2: 67731.2734375
Gradient for gab.root.butterfly_transform.butterfly_init.butterfly_matrices.3: 69984.

In [ ]:
from torch import nn, optim 

model=GABLM(128, **gab_config)
vocab_size=100
DEFAULT_CONTEXT_LENGTH=10

mock_input=torch.randn(1, DEFAULT_CONTEXT_LENGTH, 128)
mock_y=torch.randint(0, 100, (1, 10))
mock_Z={}


criterion = nn.CrossEntropyLoss()
model.train()
optimizer = optim.Adam(model.parameters())

# Zero the parameter gradients
optimizer.zero_grad()
y,X=model(mock_input, **mock_Z)
loss=nn.CrossEntropyLoss()(y.view(-1, 100), mock_y.view(-1))
loss.backward()

used_params = set()

# Hook to register which parameters are used during the forward pass
def hook_fn(module, input, output):
    for param in module.parameters():
        used_params.add(param)

# Register hooks for all submodules
hooks = []
for submodule in model.modules():
    hook = submodule.register_forward_hook(hook_fn)
    hooks.append(hook)

# Forward pass again to collect used parameters
model(mock_input)

# Remove all hooks
for hook in hooks:
    hook.remove()

all_gradients_present = True

# Check gradients
for name, param in model.named_parameters():
    if param.requires_grad:
        if param in used_params and param.grad is None:
            # Used parameter without gradient -> Error
            print(f"Error: Used parameter {name} requires gradients but has none.")
            all_gradients_present = False
        elif param not in used_params:
            # Unused parameter -> Warning
            print(f"Warning: Parameter {name} was not used in the forward pass.")

if all_gradients_present:
    print('Differentiability test passed')
else:
    print('Differentiability test failed due to missing gradients.')


Error: Used parameter gab.root.butterfly_transform.butterfly_init.butterfly_stage.butterfly_params.butterfly_weights requires gradients but has none.
Error: Used parameter gab.root.butterfly_transform.butterfly_init.butterfly_parameter.butterfly_weights requires gradients but has none.
Differentiability test failed due to missing gradients.


## Test Claude

# Read from core library

In [25]:
import sys
sys.path.append('..')

import importlib
import model_discovery.model.composer 
# importlib.reload(model_discovery.model.composer)
from model_discovery.model.composer import GAUBase, GAUTree, check_tree_name, GABComposer
from model_discovery.model.utils.modules import GABBase
import model_discovery.utils as U

# dir='/home/junyanc/model_discovery/model_discovery/model/library/core/gpt2/units'
dir='C://ChengJunyan1//Research//model_discovery//model_discovery//model//library//core//gpt2//units'
tree=GAUTree.load_from_base(dir)

print(tree.view()[0])

GAU Tree Map of GPT2:
  GPT2
       |- MHA
           |- RotaryPositionalEmbeddings
       |- GatedMLP
       |- RMSNorm

Implemented Units: MHA, GatedMLP, GPT2, RotaryPositionalEmbeddings, RMSNorm
All units are implemented.

Specifications for Implemented Units:

Unit Name: GatedMLP
'''

Gated MLP
  
'''
- Inputs: X
- Outputs: Y


Unit Name: GPT2
'''

GPT2
  
'''
- Inputs: X
- Outputs: Y


Unit Name: MHA
'''

MHA
  
'''
- Inputs: X
- Outputs: Y


Unit Name: RMSNorm
'''

RMSNorm
  
'''
- Inputs: X
- Outputs: Y


Unit Name: RotaryPositionalEmbeddings
'''

This class implements Rotary Positional Embeddings (RoPE)
proposed in https://arxiv.org/abs/2104.09864.

Reference implementation (used for correctness verfication)
can be found here:
https://github.com/meta-llama/llama/blob/main/llama/model.py#L80

In this implementation we cache the embeddings for each position upto
``max_seq_len`` by computing this during init.

Args:
    dim (int): Embedding dimension. This is usually set to the di

In [19]:
import inspect


gam_prompt_path = 'C://ChengJunyan1//Research//model_discovery//model_discovery//agents//prompts//gam_prompt.py'
gau_template_path = 'C://ChengJunyan1//Research//model_discovery//model_discovery//agents//prompts//gau_template.py'
GAM_TEMPLATE=open(gam_prompt_path).read()
GAU_TEMPLATE=open(gau_template_path).read()

GAU_BASE=inspect.getsource(GAUBase)
GAB_BASE=inspect.getsource(GABBase)
GAB_COMPOSER=inspect.getsource(GABComposer)


In [24]:
print(GAM_TEMPLATE)

from transformers.modeling_outputs import CausalLMOutput
from transformers import PreTrainedModel, PretrainedConfig
from dataclasses import dataclass

import torch
from torch import nn

from gab import GAB, gab_config


@dataclass
class GAMConfig(PretrainedConfig):
    '''Configurations for Generalized Autoregressive Models.'''

    d_model: int
    n_block: int
    batch_tokens: int 
    vocab_size: int = None


class GAM(nn.Module):
    ''' Generalized Autoregressive Models
        Input:        X: (batch, seqlen, embed_dim)
        Output:       Y: (batch, seqlen, embed_dim)
    '''
    def __init__(
        self,
        d_model: int,
        n_block: int,
        vocab_size: int = 50277,
        norm_epsilon: float = 1e-5,
        device = None,
        dtype = None,
    ) -> None:
        self.factory_kwargs = {"device": device, "dtype": dtype}
        super().__init__()
        self.d_model = d_model
        self.embedding = nn.Embedding(vocab_size, d_model, **self.factory_kwarg